In [ ]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import pydicom
import random

In [ ]:
ROOT = CONFIG.CFG.DATA.BASE
BATCH_SIZE = 64

In [ ]:
train_df = pd.read_csv(os.path.join(ROOT, "train.csv"))
train_df.head()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

In [ ]:
class OSICLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(OSICLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lc = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        out, _ = self.lstm(X)
        out = self.lc(out)
        return out

In [ ]:
model = OSICLSTM(4, 100, 1, 4)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
len(list(model.parameters()))

In [ ]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

In [ ]:
print(model)

In [ ]:
ALL_PATIENTS = train_df["Patient"].unique().tolist()

In [ ]:
model.train()

In [ ]:
for patient in ALL_PATIENTS:
    patient_data = train_df[train_df["Patient"] == patient][['Weeks', 'FVC', 'Percent', 'Age']].values
    patient_data = torch.tensor(patient_data).float()

    for data in patient_data:
        out = model(data.view(1,1,-1))
        print(out)
        break
    break

In [ ]:
for epoch in range(1000):
    total_loss = 0
    for patient in ALL_PATIENTS:
        model.zero_grad()

        patient_data = train_df[train_df["Patient"] == patient][['Weeks', 'FVC', 'Percent', 'Age']].values
        patient_data = torch.tensor(patient_data).float()

        loss = 0

        for i in range(patient_data.size()[0] - 1):
            out = model(data.view(1,1,-1))
            loss += loss_function(out, patient_data[i+1])
            total_loss += loss.item()
        # for data in patient_data:
        #     out = model(data.view(1, 1, -1))
        #     print(out)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, loss={total_loss}")

In [ ]:
sample = torch.tensor([-8, 2698.674521,   60,   67.7956])
model(sample.view(1,1,-1))

In [ ]:
for i in range(5):
    print(sample)
    out = model(sample.view(1, 1, -1))
    print(out)
    sample = out

In [ ]:
with torch.no_grad():
    input = torch.randn(2, 5, 4)
    print(input)
    print()
    out = model(input)
    print(out)
    print(out.shape)